In [ ]:
from spade.behaviour import CyclicBehaviour
from spade.message import Message
from transformers import pipeline
agent1.add_behaviour(ThreadMonitor())

toxic = pipeline("text-classification", model="unitary/toxic-bert", return_all_scores=True)

def __init__(self, *args, **kwargs):
    super().__init__(*args, **kwargs)
    # Map user JID to list of past scores

class ThreadMonitor(CyclicBehaviour):
    async def run(self):
        # Wait for incoming XMPP messages
        msg: Message = await self.receive(timeout=30)
        if msg:
            text = msg.body
            sender = str(msg.sender)
            await self.agent.bdi.set_belief("message", text, sender)
            await self.agent.bdi.post_event("!python_analyze", text, sender)
           
@agent1.agent.bdi.on_event("!python_analyze")
async def analyze_message(self, text, sender):
    score = classify_text(text)
    msg_id = f"{hash(text)}_{sender}"
    self.agent.bdi.set_belief("harmful", msg_id, text, score)
    msg1 = Message(to="grp27.3.cpt302@xmpp.jp")
    msg1.set_metadata("performative", "inform")
    
    msg1.body = f"harmful({msg_id},{text}, {score}, {sender})"
    await self.send(msg1)

    msg2 = Message(to="grp27.2.cpt302@xmpp.jp")
    msg2.set_metadata("performative", "inform")
    msg_id = f"{hash(text)}_{str(msg.sender)}"
    msg2.body = f"harmful({msg_id},{text}, {score})"
    await self.send(msg2)
    return True


def classify_text(text):
    # Use the toxic pipeline to classify the text
    results = toxic(text)
    # Unwrap nested list if return_all_scores=True
    entries = results[0] if isinstance(results, list) and isinstance(results[0], list) else results
    # Extract the scores for each label
    scores = {item['label']: item['score'] for item in entries}

    # Define weights for each label to reflect severity
    weights = {
        'toxic': 1.0,
        'severe_toxic': 0,
        'obscene': 1.0,
        'insult': 1.0,
        'identity_hate': 0,
    }
    # Compute weighted average harmfulness
    weighted_sum = sum(scores.get(label, 0.0) * weights.get(label, 1.0) for label in weights)
    total_weight = sum(weights.values())
    avg_harm = weighted_sum / total_weight

    # Map [0.0…1.0] avg_harm to [-1.0…+1.0] composite scale
    composite = 1.0 - avg_harm * 2.0
    return composite


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
c:\Users\jason\anaconda3\envs\myenv\lib\site-packages\transformers\pipelines\text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
c:\Users\jason\anaconda3\envs\myenv\lib\site-packages\transformers\pipelines\text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [17]:
# samples = [
#     "You're such an idiot, nobody cares about your opinion",
#     "I think your point makes sense, but maybe we could also consider this other perspective"
# ]
# for text in samples:
#     result = classify_text(text)
#     print(f"Input: {text}\nClassification scores: {result}\n")



text = "You're such an idiot, nobody cares about your opinion"
print(classify_text(text))

-0.6232570012410481
